# 教學目標

- 複習 PTT 文章的爬蟲邏輯
- 熟悉單網站多網頁，透過列表將連結內的文章內容都爬下來

In [4]:
import requests
import re
import json
from urllib.parse import urljoin
from bs4 import BeautifulSoup

# PTT 八卦版網址
PTT_URL = 'https://www.ptt.cc/bbs/Gossiping/index.html'

## 範例重點

這個練習題在 PTT 單頁文章上的程式碼比較多，建議可以先把他寫成一個 function 方便後面程式邏輯的撰寫

In [5]:
def crawl_article(url):
    response = requests.get(url, cookies={'over18': '1'})
    
    # 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
    if response.status_code != 200:
        print('Error - {} is not available to access'.format(url))
        return
    
    # 將網頁回應的 HTML 傳入 BeautifulSoup 解析器, 方便我們根據標籤 (tag) 資訊去過濾尋找
    soup = BeautifulSoup(response.text)
    
    # 取得文章內容主體
    main_content = soup.find(id='main-container')
    
    # 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
    metas = main_content.select('div.article-metaline')
    author = ''
    title = ''
    date = ''
    if metas:
        if metas[0].select('span.article-meta-value')[0]:
            author = metas[0].select('span.article-meta-value')[0].string
        if metas[1].select('span.article-meta-value')[0]:
            title = metas[1].select('span.article-meta-value')[0].string
        if metas[2].select('span.article-meta-value')[0]:
            date = metas[2].select('span.article-meta-value')[0].string

        # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
        #
        # .extract() 方法可以參考官方文件
        #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
        for m in metas:
            m.extract()
        for m in main_content.select('div.article-metaline-right'):
            m.extract()
    
    # 取得留言區主體
    pushes = main_content.find_all('div', class_='push')
    for p in pushes:
        p.extract()
    
    # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
    # 透過 regular expression 取得 IP
    # 因為字串中包含特殊符號跟中文, 這邊建議使用 unicode 的型式 u'...'
    try:
        ip = main_content.find(text=re.compile(u'※ 發信站:'))
        ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
    except Exception as e:
        ip = ''
    
    # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
    # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
    #
    # 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字, 可參考官方文件 
    #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
    filtered = []
    for v in main_content.stripped_strings:
        # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
        if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
            filtered.append(v)

    # 定義一些特殊符號與全形符號的過濾器
    expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
    for i in range(len(filtered)):
        filtered[i] = re.sub(expr, '', filtered[i])
    
    # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
    filtered = [i for i in filtered if i]
    content = ' '.join(filtered)
    
    # 處理留言區
    # p 計算推文數量
    # b 計算噓文數量
    # n 計算箭頭數量
    p, b, n = 0, 0, 0
    messages = []
    for push in pushes:
        # 假如留言段落沒有 push-tag 就跳過
        if not push.find('span', 'push-tag'):
            continue
        
        # 過濾額外空白與換行符號
        # push_tag 判斷是推文, 箭頭還是噓文
        # push_userid 判斷留言的人是誰
        # push_content 判斷留言內容
        # push_ipdatetime 判斷留言日期時間
        push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
        push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
        push_content = push.find('span', 'push-content').strings
        push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
        push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')

        # 整理打包留言的資訊, 並統計推噓文數量
        messages.append({
            'push_tag': push_tag,
            'push_userid': push_userid,
            'push_content': push_content,
            'push_ipdatetime': push_ipdatetime})
        if push_tag == u'推':
            p += 1
        elif push_tag == u'噓':
            b += 1
        else:
            n += 1
    
    # 統計推噓文
    # count 為推噓文相抵看這篇文章推文還是噓文比較多
    # all 為總共留言數量 
    message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}
    
    # 整理文章資訊
    data = {
        'url': url,
        'article_author': author,
        'article_title': title,
        'article_date': date,
        'article_content': content,
        'ip': ip,
        'message_count': message_count,
        'messages': messages
    }
    return data

## 範例重點

分析列表取得要爬蟲的網址

In [21]:
# 對文章列表送出請求並取得列表主體
resp = requests.get(PTT_URL, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)
main_list = soup.find('div', class_='bbs-screen')
all_data = []

# 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
for div in main_list.findChildren('div', recursive=False):
    class_name = div.attrs['class']
    
    # 遇到分隔線要處理的情況
    if class_name and 'r-list-sep' in class_name:
        print('Reach the last article')
        break
    # 遇到目標文章
    if class_name and 'r-ent' in class_name:
        div_title = div.find('div', class_='title')
        if div_title.text[10:16]=='本文已被刪除':
            continue
        else:
            a_title = div_title.find('a', href=True)
            
            article_URL = urljoin(PTT_URL, a_title['href'])
            article_title = a_title.text
            print('Parse {} - {}'.format(article_title, article_URL))
        
            # 呼叫上面寫好的 function 來對文章進行爬蟲
            parse_data = crawl_article(article_URL)
        
            # 將爬完的資料儲存
            all_data.append(parse_data)

Parse [問卦] 為什麼馮提莫那麼可愛?? 屌打台灣藝人!!! - https://www.ptt.cc/bbs/Gossiping/M.1580256713.A.922.html
Parse [新聞] 買口罩大作戰 記者凌晨實測6家超商3家售 - https://www.ptt.cc/bbs/Gossiping/M.1580256738.A.A41.html
Parse [問卦] 所以現在有沒有禁止戴口罩？ - https://www.ptt.cc/bbs/Gossiping/M.1580256743.A.B5C.html
Parse Re: [新聞] 突發：白宮表示考慮取消中美間所有航班 - https://www.ptt.cc/bbs/Gossiping/M.1580256805.A.A5C.html
Parse [爆卦] 日本武漢包機 產地直送2人發燒 - https://www.ptt.cc/bbs/Gossiping/M.1580256907.A.B03.html
Parse [問卦] 專門製造五樓梗來亂推文的八卦 - https://www.ptt.cc/bbs/Gossiping/M.1580256987.A.D41.html
Parse [問卦] 軍服剪裁還是真的大？ - https://www.ptt.cc/bbs/Gossiping/M.1580257041.A.B32.html
Parse [問卦] 早上咳嗽一堆人在看我XDDD - https://www.ptt.cc/bbs/Gossiping/M.1580257298.A.788.html
Parse [問卦] 為了錢願意去舔共嗎 - https://www.ptt.cc/bbs/Gossiping/M.1580257317.A.DFD.html
Parse [問卦] Kobe 之死早已預言 ？ - https://www.ptt.cc/bbs/Gossiping/M.1580257326.A.62C.html
Parse [新聞] 兒子縱火燒自家彩券行 母不治身亡 - https://www.ptt.cc/bbs/Gossiping/M.1580257542.A.DFB.html
Parse [問卦] 真正有錢人都怎麼藏錢的 - https://www.ptt.cc/bbs/Gos

In [22]:
# 將爬完的資訊存成 json 檔案
with open('parse_data.json', 'w+') as f:
    json.dump(all_data, f, ensure_ascii=False, indent=4)

In [23]:
import pandas as pd
pd.DataFrame(all_data)

,article_author,article_content,article_date,article_title,ip,message_count,messages,url
0,assassinASHE (幹古專用帳號),這幾天在油管翻了好多支她的視頻\n\nOMG\n\n馮提莫怎麼那麼可愛啦\n\n看一支影片就...,Wed Jan 29 08:11:50 2020,[問卦] 為什麼馮提莫那麼可愛?? 屌打台灣藝人!!!,27.105.57.177,"{'all': 12, 'count': 0, 'push': 4, 'boo': 4, '...","[{'push_tag': '→', 'push_userid': 'ununnihao',...",https://www.ptt.cc/bbs/Gossiping/M.1580256713....
1,redspeed (RED),1.媒體來源: UDN 聯合新聞網\n\n2.記者署名 20200129 04:24聯合報 ...,Wed Jan 29 08:12:13 2020,[新聞] 買口罩大作戰 記者凌晨實測6家超商3家售,1.164.109.18,"{'all': 26, 'count': 4, 'push': 6, 'boo': 2, '...","[{'push_tag': '→', 'push_userid': 'ununnihao',...",https://www.ptt.cc/bbs/Gossiping/M.1580256738....
2,deathsong (林義雄殺了北極熊),前幾年因為硬關核電廠 明明就缺電\n\n還嘴硬說不是缺電只是需要節電 搞一個大停電出來\n\...,Wed Jan 29 08:12:21 2020,[問卦] 所以現在有沒有禁止戴口罩？,111.241.46.165,"{'all': 14, 'count': -5, 'push': 1, 'boo': 6, ...","[{'push_tag': '→', 'push_userid': 'zold', 'pus...",https://www.ptt.cc/bbs/Gossiping/M.1580256743....
3,neil136 (暱稱好難),拜託臺灣跟進啊，想想Sars的狀況\n現任政府直接一樣方法做下去啦\n幹，2024管你推誰出...,Wed Jan 29 08:13:23 2020,Re: [新聞] 突發：白宮表示考慮取消中美間所有航班,115.82.148.184,"{'all': 10, 'count': -1, 'push': 0, 'boo': 1, ...","[{'push_tag': '→', 'push_userid': 'zold', 'pus...",https://www.ptt.cc/bbs/Gossiping/M.1580256805....
4,taotzu (╮(╯▽╰),https://i.imgur.com/Mh7juns.jpg 日本武漢包機共搭載206名\...,Wed Jan 29 08:15:05 2020,[爆卦] 日本武漢包機 產地直送2人發燒,1.200.49.188,"{'all': 66, 'count': 36, 'push': 37, 'boo': 1,...","[{'push_tag': '→', 'push_userid': 'zold', 'pus...",https://www.ptt.cc/bbs/Gossiping/M.1580256907....
5,v89074 (Vincent),如題，最近發現最近武漢善意散播全球。開始有些網路工作者（五毛）開始繼續使用資訊洗腦，或是回覆...,Wed Jan 29 08:16:23 2020,[問卦] 專門製造五樓梗來亂推文的八卦,172.58.95.10,"{'all': 8, 'count': 0, 'push': 1, 'boo': 1, 'n...","[{'push_tag': '→', 'push_userid': 'ununnihao',...",https://www.ptt.cc/bbs/Gossiping/M.1580256987....
6,mossdevin (Devin Moss 10),朋友傳了一張圖片\n因為太突出了\n所以想問\n\n右邊是天賦異稟呢？\n還是純粹褲子剪裁的...,Wed Jan 29 08:17:19 2020,[問卦] 軍服剪裁還是真的大？,180.217.160.242,"{'all': 5, 'count': 2, 'push': 2, 'boo': 0, 'n...","[{'push_tag': '→', 'push_userid': 'ununnihao',...",https://www.ptt.cc/bbs/Gossiping/M.1580257041....
7,assassinASHE (幹古專用帳號),笑死\n\n開工第一天\n\n騎著小餔餔停在平交道口\n\n火車過去的時候 不知道是灰塵飛起...,Wed Jan 29 08:21:35 2020,[問卦] 早上咳嗽一堆人在看我XDDD,27.105.57.177,"{'all': 8, 'count': 1, 'push': 3, 'boo': 2, 'n...","[{'push_tag': '噓', 'push_userid': 'ununnihao',...",https://www.ptt.cc/bbs/Gossiping/M.1580257298....
8,ozy14401 (咪寶來了D好，但是說了四),我就問一個問題\n\n各位鄉民 假如你是藝人\n\n為了能賺更多錢 會不會舔中\n\n請誠實...,Wed Jan 29 08:21:55 2020,[問卦] 為了錢願意去舔共嗎,223.140.199.85,"{'all': 15, 'count': 6, 'push': 6, 'boo': 0, '...","[{'push_tag': '推', 'push_userid': 'neil136', '...",https://www.ptt.cc/bbs/Gossiping/M.1580257317....
9,currry (南港李國毅),如題\n\nKobe到了生命的盡頭\n\n依舊是充滿話題及傳奇的人物\n\n最後一戰\n\n...,Wed Jan 29 08:22:04 2020,[問卦] Kobe 之死早已預言 ？,110.28.8.120,"{'all': 13, 'count': 6, 'push': 8, 'boo': 2, '...","[{'push_tag': '→', 'push_userid': 'ununnihao',...",https://www.ptt.cc/bbs/Gossiping/M.1580257326....
